# MRU Base model
## Evaluate per user

In [1]:
#import preprocessed per user dataset from onedrive prickled file
import pickle
import os
import numpy as np

path = r"C:/Users/natha/OneDrive - The University of Sydney (Students)/CS48-CAPSTONE Project 2021 Sem1/dataset"
#create output dictionary
data = {}

N_user = 1000

for u in os.listdir(path+'/user_preprocessed_pickle')[:N_user]: #import 100users

    uid = int(u[:-7])
    file_name = '/'.join([path,'/user_preprocessed_pickle', u])

    try:
        with open(file_name, 'rb') as f:
            dic = pickle.load(f)
            data[uid] = dic[uid]
    except:
        print(dic)
total_user = len(data.keys())
print("Imported {} users".format(total_user))

Imported 870 users


In [2]:
#create 2 function to extract the feature and label from array
def extract_time(lst):
    #extract first columns (session) from ROWS
    return [item[:1] for item in lst]

def extrac_place(lst):
    #extract second columns (mode base_station) from ROWS
    return [item[1] for item in lst]

def extract_app(lst):
    #extract app_usage columns from ROWS
    return np.array([item[2:] for item in lst])

# function to generate input for baseline model
def generate_baseline_input(data):

    data_neural = data

    all_prev_app = []
    all_target_app = []
    all_prev_t = []
    all_target_t = []

    day_id = [26] #using all the days for training

    # if candidate is None:
    candidate = data_neural.keys() #filter, and get user id

    #iterate all the users
    for u in candidate:
        #seperate feature and label list
        #get user's record
        sessions = data_neural[u]
        #sepearate to store pred and true
        user_prev_app = []
        user_target_app = []
        user_prev_t = []
        user_target_t = []

        for i in day_id:
            #call specific day
            session = sessions[i] #= data[u][i]
            # session is (48,2002)
            # idea here is to iterate over all 48 rows to extract the first columns and last 2000 columns

            #extract app_usage
            app_usage = np.array(extract_app(session))
            #extract time
            time_usage = np.array(extract_time(session))

            #slice the app section to get app usage of ind 0 to second last rows (47,2000)
            prev_app = np.array(app_usage)[:-1,]
            #slice the app section to get app usage of ind 1 to last rows (47,2000)
            target_app = np.array(app_usage)[1:,]

            prev_t = np.array(time_usage)[:-1,]
            target_t = np.array(time_usage)[1:,]

            user_prev_app.append(prev_app)
            user_target_app.append(target_app)
            user_prev_t.append(prev_t)
            user_target_t.append(target_t)
            # print(current_session[24])
            # print(previous_session[24])
        #after iterate the day, need to shift the feature columns upwards for prediction
        #also need to cut the last rows for labels
        #reshape: as orginal dataset were seperate by day,session,column:
        #feature:4(days),48sessions,2features
        #label:4(days),48sessions,2000 app usage(count)
        #reshape to flattern the data
        # user_prev = np.array(user_prev).reshape(-1,2000)
        # user_true = np.array(user_true).reshape(-1,2000)
        # print(user_prev[24])
        # print(user_true[24])
        #append per user's feature and label to all_user list
        all_prev_app.append(user_prev_app)
        all_target_app.append(user_target_app)
        all_prev_t.append(user_prev_t)
        all_target_t.append(user_target_t)

    all_prev_app= np.array(all_prev_app)
    all_target_app = np.array(all_target_app)
    all_prev_t = np.array(all_prev_t)
    all_target_t = np.array(all_target_t)

    return all_prev_app, all_target_app, all_prev_t, all_target_t

In [3]:
app_np, app_target, tim_np, ptim_np = generate_baseline_input(data)
print(app_np.shape, app_target.shape, tim_np.shape, ptim_np.shape)

(870, 1, 47, 2000) (870, 1, 47, 2000) (870, 1, 47, 1) (870, 1, 47, 1)


In [4]:
app_np = app_np.reshape(total_user,-1,2000)
app_target = app_target.reshape(total_user,-1,2000)
tim_np = tim_np.reshape(total_user,-1,1)
ptim_np = ptim_np.reshape(total_user,-1,1)
print(app_np.shape, app_target.shape, tim_np.shape, ptim_np.shape)

(870, 47, 2000) (870, 47, 2000) (870, 47, 1) (870, 47, 1)


In [5]:
#metrics function

from sklearn import metrics as skmetrics

def cal_ap( y_actual, y_pred, k ):
    topK = min( len(y_pred), k ) # set top k
    l_zip = list(zip(y_actual,y_pred))
    # sort y_pred by the probability of the model
    s_zip = sorted( l_zip, key=lambda x: x[1], reverse=True )
    # topk of sorted result
    s_zip_topk = s_zip[:topK] # Shape (5,2)
    # Calculation of precision
    num = 0
    rank = 0
    sumP = 0.0
    for item in s_zip_topk:
        rank += 1
        if item[0] == 1:
            num += 1
            sumP += (num*1.0)/(rank*1.0)
    ap = 0.0
    if num > 0:
        ap = sumP/(num*1.0)
    return ap   # average precision
# Take topk prediction and the ground truth

def r_k(y_actual, y_pred, k, threshold):
    topK = min( len(y_pred), k ) # set top k
    l_zip = list(zip(y_actual,y_pred))
    # sort y_pred by the probability of the model
    s_zip = sorted( l_zip, key=lambda x: x[1], reverse=True )
    # topk of sorted result
    s_zip_topk = s_zip[:topK] # Shape (5,2)
    # print(s_zip_topk)
    actual, pred = zip(*s_zip_topk)
    actual = np.where(np.array(actual) > threshold, 1, 0)
    pred = np.array(pred)
    pred_o = np.where(pred > threshold, 1, 0)
    return actual, pred_o, pred


def user_evaluation_metrics(y_pred, y_test):
    total_auc = 0
    total_map = 0
    total_recall = 0
    v_count = 0
    count = 0

    y_pred = np.where(y_pred >0,1,0)
    y_test = np.where(y_test >0,1,0)
    # print(y_pred.shape)
    # print(y_test.shape)

    for i in range(y_test.shape[0]):

        if (np.sum(y_test[i])> 0):
            fpr, tpr, thresholds = skmetrics.roc_curve(y_test[i], y_pred[i], pos_label=1)
            total_auc += skmetrics.auc(fpr, tpr)
            actual, pred_o, pred = r_k(y_test[i], y_pred[i] ,5, 0.5)
            total_recall += skmetrics.recall_score(actual, pred_o, average='macro') # Recall@5
            total_map += cal_ap(y_test[i], y_pred[i], 2000)
            #divide only the valid rows that has data
            v_count +=1

        else:
            count +=1
            pass
            
    if v_count != 0:
        total_auc = total_auc / v_count
        total_map = total_map / v_count
        total_recall = total_recall / v_count
    else:
        pass

    return total_auc, total_map, total_recall, v_count, count
        
    # print('AUC: ', avg_auc / app_target.shape[0])
    # print('MAP: ', avg_map / app_target.shape[0])
    # print('Recall@5: ', avg_recall / app_target.shape[0])
    # print('Skipped: {} rows, total: {} rows'.format(count, app_target.shape[0]))

# fpr, tpr, thresholds = skmetrics.roc_curve(y_test, pred_MNB_prob, pos_label=1) # Collect the recall and false positive rate from all 2000 predictions
# acc[0] += skmetrics.auc(fpr, tpr)
# # acc[1] += cal_ap(truth, predict, 5)

In [6]:
from tqdm.notebook import tqdm
total_user_auc = 0
total_user_map = 0
total_user_recall = 0
total_valid_rows = 0
total_non_valid_rows = 0

for i in tqdm(range(total_user)):
    app_score = np.zeros([len(app_target[i]), 2000])
    for t in range(len(tim_np[i])):
        if(ptim_np[i][t] == (tim_np[i][t]+1)):
            app_score[t] = app_np[i][t]
    # print(app_score.shape, app_target.shape)
    # print(app_score.shape, app_target[i].shape)
    # print(len(app_target[i]))
    user_auc, user_map, user_recall, valid_rows, non_valid_rows = user_evaluation_metrics(app_score, app_target[i])

    total_user_auc+= user_auc
    total_user_map+= user_map
    total_user_recall+= user_recall

    total_valid_rows += valid_rows
    total_non_valid_rows += non_valid_rows

print('Total user: ', total_user)
print('AUC: ', total_user_auc / total_user)
print('MAP: ', total_user_map / total_user)
print('Recall@5: ', total_user_recall / total_user)
print('Rows: {} / {}'.format(total_valid_rows, (total_valid_rows + total_non_valid_rows)))


            # print(app_np[i].shape)
            # print(app_target[i].shape)
            # print(tim_np[i].shape)
            # print(ptim_np[i].shape)

C:\Users\natha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

Total user:  870
AUC:  0.563273208459653
MAP:  0.2847653478424465
Recall@5:  0.5858401073724266
Rows: 11422 / 40890


# MRU
## Evaluate whole dataset

In [7]:
#import preprocessed per user dataset from onedrive prickled file
import pickle
import os
import numpy as np

path = r"C:/Users/natha/OneDrive - The University of Sydney (Students)/CS48-CAPSTONE Project 2021 Sem1/dataset"
#create output dictionary
data = {}

N_user = 1000

for u in os.listdir(path+'/user_preprocessed_pickle')[:N_user]: #import 100users

    uid = int(u[:-7])
    file_name = '/'.join([path,'/user_preprocessed_pickle', u])

    try:
        with open(file_name, 'rb') as f:
            dic = pickle.load(f)
            data[uid] = dic[uid]
    except:
        print(dic)

print("Imported {} users".format(len(data.keys())))

Imported 870 users


In [8]:
#create 2 function to extract the feature and label from array
def extract_time(lst):
    #extract first 2 columns: session and mode location 
    return [item[:1] for item in lst]

def extract_app(lst):
    #extract the remaning columns
    return np.array([item[2:] for item in lst])

# function to generate input for baseline model
def generate_baseline_input(data):

    data_neural = data

    all_prev_app = []
    all_target_app = []
    all_prev_t = []
    all_target_t = []

    day_id = [26] #using all the days for training

    # if candidate is None:
    candidate = data_neural.keys() #filter, and get user id

    #iterate all the users
    for u in candidate:
        #seperate feature and label list
        #get user's record
        sessions = data_neural[u]
        #sepearate to store pred and true
        user_prev_app = []
        user_target_app = []
        user_prev_t = []
        user_target_t = []

        for i in day_id:
            #call specific day
            session = sessions[i]
            #append feature to feature list, label to label list
            app_usage = np.array(extract_app(session))
            time_usage = np.array(extract_time(session))
            # print(app_usage)
            #basically just just the previous session as the "pred" of next session
            prev_app = np.array(app_usage)[:-1,]
            target_app = np.array(app_usage)[1:,]

            prev_t = np.array(time_usage)[:-1,]
            target_t = np.array(time_usage)[1:,]

            user_prev_app.append(prev_app)
            user_target_app.append(target_app)
            user_prev_t.append(prev_t)
            user_target_t.append(target_t)
            # print(current_session[24])
            # print(previous_session[24])
        #after iterate the day, need to shift the feature columns upwards for prediction
        #also need to cut the last rows for labels
        #reshape: as orginal dataset were seperate by day,session,column:
        #feature:4(days),48sessions,2features
        #label:4(days),48sessions,2000 app usage(count)
        #reshape to flattern the data
        # user_prev = np.array(user_prev).reshape(-1,2000)
        # user_true = np.array(user_true).reshape(-1,2000)
        # print(user_prev[24])
        # print(user_true[24])
        #append per user's feature and label to all_user list
        all_prev_app.append(user_prev_app)
        all_target_app.append(user_target_app)
        all_prev_t.append(user_prev_t)
        all_target_t.append(user_target_t)

    all_prev_app= np.array(all_prev_app)
    all_target_app = np.array(all_target_app)
    all_prev_t = np.array(all_prev_t)
    all_target_t = np.array(all_target_t)

    return all_prev_app, all_target_app, all_prev_t, all_target_t

In [9]:
app_np, app_target, tim_np, ptim_np = generate_baseline_input(data)
print(app_np.shape, app_target.shape, tim_np.shape, ptim_np.shape)

(870, 1, 47, 2000) (870, 1, 47, 2000) (870, 1, 47, 1) (870, 1, 47, 1)


In [10]:
app_np = app_np.reshape(-1,2000)
app_target = app_target.reshape(-1,2000)
# app_np = np.where(app_np > 0, 1,0)
# app_target = np.where(app_target > 0, 1,0)
print(app_np.shape)
print(app_target.shape)

(40890, 2000)
(40890, 2000)


In [11]:
from sklearn import metrics as skmetrics
from tqdm.notebook import tqdm
def cal_ap( y_actual, y_pred, k ):
    topK = min( len(y_pred), k ) # set top k
    l_zip = list(zip(y_actual,y_pred))
    # sort y_pred by the probability of the model
    s_zip = sorted( l_zip, key=lambda x: x[1], reverse=True )
    # topk of sorted result
    s_zip_topk = s_zip[:topK] # Shape (5,2)
    # Calculation of precision
    num = 0
    rank = 0
    sumP = 0.0
    for item in s_zip_topk:
        rank += 1
        if item[0] == 1:
            num += 1
            sumP += (num*1.0)/(rank*1.0)
    ap = 0.0
    if num > 0:
        ap = sumP/(num*1.0)
    return ap   # average precision
# Take topk prediction and the ground truth

def r_k(y_actual, y_pred, k, threshold):
    topK = min( len(y_pred), k ) # set top k
    l_zip = list(zip(y_actual,y_pred))
    # sort y_pred by the probability of the model
    s_zip = sorted( l_zip, key=lambda x: x[1], reverse=True )
    # topk of sorted result
    s_zip_topk = s_zip[:topK] # Shape (5,2)
    # print(s_zip_topk)
    actual, pred = zip(*s_zip_topk)
    actual = np.where(np.array(actual) > threshold, 1, 0)
    pred = np.array(pred)
    pred_o = np.where(pred > threshold, 1, 0)
    return actual, pred_o, pred


def user_evaluation_metrics(y_pred, y_test):
    total_auc = 0
    total_map = 0
    total_recall = 0
    v_count = 0
    count = 0

    y_pred = np.where(y_pred >0,1,0)
    y_test = np.where(y_test >0,1,0)
    for i in tqdm(range(y_test.shape[0])):
        if (np.sum(y_test[i])> 0):
            fpr, tpr, thresholds = skmetrics.roc_curve(y_test[i], y_pred[i], pos_label=1)
            total_auc += skmetrics.auc(fpr, tpr)
            actual, pred_o, pred = r_k(y_test[i], y_pred[i] ,5, 0.5)
            total_recall += skmetrics.recall_score(actual, pred_o, average='macro') # Recall@5
            total_map += cal_ap(y_test[i], y_pred[i], len(y_test[i]))
            v_count +=1
        else:
            count +=1
            pass
        
    total_auc = total_auc / v_count
    total_map = total_map / v_count
    total_recall = total_recall / v_count

    return total_auc, total_map, total_recall, v_count, count
        
    # print('AUC: ', avg_auc / app_target.shape[0])
    # print('MAP: ', avg_map / app_target.shape[0])
    # print('Recall@5: ', avg_recall / app_target.shape[0])
    # print('Skipped: {} rows, total: {} rows'.format(count, app_target.shape[0]))

# fpr, tpr, thresholds = skmetrics.roc_curve(y_test, pred_MNB_prob, pos_label=1) # Collect the recall and false positive rate from all 2000 predictions
# acc[0] += skmetrics.auc(fpr, tpr)
# # acc[1] += cal_ap(truth, predict, 5)


In [12]:
a,m,r,v,c= user_evaluation_metrics(app_np, app_target)

print('AUC: ', a)
print('MAP: ', m)
print('Recall@5: ', r)
print("Rows: {} / {}".format(v,v+c))


AUC:  0.6640623867305869
MAP:  0.34415445719949234
Recall@5:  0.6419585011381499
Rows: 11422 / 40890
